# Analysis of the Nissl volume expression

## Description

The mouse brain reference volumes are a pair of volumetric files from the Allen Institute for Brain Science (AIBS): the **Nissl** and the **annotations** volumes.

The **Nissl volume** shows every cell in the brain. It is used to obtain the cell distribution in the mouse brain.
The **annotations volume** provides a brain region id to each voxel of the Nissl volume and any volumetric dataset of the AIBS. The mouse brain regions are organized in a hierarchical tree, with leaves representing the finest subdivisions captured by the AIBS. This **hierarchy** is stored in a **json file** and is common to each reference atlas version. It is also available on the AIBS website.

These two whole brain volumes exist in multiple versions and need to be aligned to each other to produce the best results in terms of cell density estimation (see https://doi.org/10.1101/2021.11.20.469384 for more details).

The Nissl volume is derived from coronal slice image stacks from a whole brain Nissl staining experiment (see Dong 2007). The Nissl volume has therefore artefacts for the coronal slicing such as holes, tearing and stretching. There are also some slices that were under- or over-exposed when compared to other slices.

Here we are plotting the distribution of the Nissl volume expression on the sagittal plane to highlight the differences in exposition of the different coronal slices.

In [ ]:
import numpy as np
from voxcell import RegionMap, VoxelData
from os.path import join
from matplotlib.pylab import plt

Set below the path to the folder containing the data, including the annotations volumes, the Nissl volumes and the json hierarchy file.

Set also the folder in which the figures produced will be stored.

In [ ]:
DATA_FOLDER = 'path/to/data/'
OUTPUT_FOLDER = "path/to/output"

In [ ]:
region_map = RegionMap.load_json(join(DATA_FOLDER, "brain_regions.json"))
annotation = VoxelData.load_nrrd(join(DATA_FOLDER, "annotations.nrrd")).raw
nissl = VoxelData.load_nrrd(join(DATA_FOLDER, "ara_nissl_25.nrrd")).raw

## Set matplotlib parameters

In [ ]:
SMALL_SIZE = 48
MEDIUM_SIZE = 48
BIGGER_SIZE = 48
plt.rcParams["font.family"] = "Arial"
plt.rc('font', size=SMALL_SIZE)          # controls default text sizes
plt.rc('axes', titlesize=BIGGER_SIZE)     # fontsize of the axes title
plt.rc('axes', labelsize=MEDIUM_SIZE)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('ytick', labelsize=SMALL_SIZE)    # fontsize of the tick labels
plt.rc('legend', fontsize=SMALL_SIZE)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

plt.rcParams['xtick.major.pad']  = 9.0
plt.rcParams['axes.linewidth'] = 2.0
plt.rcParams['xtick.major.size' ] = 7.0*3.0
plt.rcParams['xtick.minor.size' ] = 4.0*3.0
plt.rcParams['ytick.major.size' ] = 7.0*3.0
plt.rcParams['ytick.minor.size' ] = 4.0*3.0
plt.rcParams['xtick.major.width'] = 2.4
plt.rcParams['xtick.minor.width'] = 1.8
plt.rcParams['ytick.major.width'] = 2.4
plt.rcParams['ytick.minor.width'] = 1.8

# color dictionary
CBcdict={
    'Bl':np.array([0,0,0]),
    'Or':np.array([.9,.6,0]),
    'SB':np.array([.35,.7,.9]),
    'bG':np.array([0,.6,.5]),
    'Ye':np.array([.95,.9,.25]),
    'Bu':np.array([0,.45,.7]),
    'Ve':np.array([.8,.4,0]),
    'rP':np.array([.8,.6,.7]),
}

## Plot the median nissl value along the sagittal plane

### Sagittal plot of the original Nissl volume for comparison.

In [ ]:
slice_ = 265 # int(annotation.shape[2] // 2.5)
cmap = plt.cm.gray
norm = plt.Normalize(nissl.min(), nissl.max())
rgba = cmap(1-norm(nissl[:, :, slice_].T))

rgba-=0.1
rgba = np.maximum(rgba/np.max(rgba), 0)
fig = plt.figure(figsize=(15, 9.1))
plt.imshow(rgba, interpolation='nearest')

plt.axis('off')
plt.xlim([0, annotation.shape[0]])
plt.ylim([annotation.shape[1], 0])
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "sagittal_nissl.png"), dpi=200)

### Sagittal plot of the median expression of the Nissl volume

In [ ]:
filt = np.where(annotation!=0)
x_pos = np.unique(filt[0])

hist_nissl = np.zeros(len(x_pos))
nissl_med = np.zeros((annotation.shape[0], annotation.shape[1], 1))
for i, x in enumerate(x_pos):
    subfilt = np.where(filt[0]==x)
    hist_nissl[i] = np.median(nissl[x, filt[1][subfilt], filt[2][subfilt]])
    for y in range(annotation.shape[1]):
        loc_filt = nissl[x][y]
        loc_ann = annotation[x][y] != 0
        if np.any(loc_ann):
            nissl_med[x][y][0] = np.median(loc_filt[loc_ann])

There are three major changes in Nissl expression that seem to be related to change in exposure of the coronal nissl slices.

In [ ]:
# exposure_change_slices = [119, 240, 360] # positions for the CCFbbp Nissl volume
exposure_change_slices = [110, 230, 350] # for the CCFv2 / CCFv3 Nissl volume

In [ ]:
cmap = plt.cm.gray
norm = plt.Normalize(nissl_med.min(), nissl_med.max())
rgba = cmap(1-norm(nissl_med[:, :, 0].T))

rgba -= 0.1
rgba = np.maximum(rgba/np.max(rgba), 0)
fig = plt.figure(figsize=(15, 9.1))
plt.imshow(rgba, interpolation='nearest')

# Show the positions of the changes in Nissl expression
plt.axvline(x=[exposure_change_slices[0]],
            ymin=(np.max(np.where(nissl_med[exposure_change_slices[0],
                                            :, 0]>0))) / annotation.shape[1], 
            color=CBcdict["Bu"], linewidth=6)
plt.axvline(x=[exposure_change_slices[0]], 
            ymax=(np.min(np.where(nissl_med[exposure_change_slices[0],
                                            :, 0]>0)))/annotation.shape[1], 
            color=CBcdict["Bu"], linewidth=6)

plt.axvline(x=[exposure_change_slices[1]],
            ymin=(np.max(np.where(nissl_med[exposure_change_slices[1],
                                            :, 0]>0)) + 10) / annotation.shape[1], 
            color=CBcdict["Or"], linewidth=6)
plt.axvline(x=[exposure_change_slices[1]], 
            ymax=(np.min(np.where(nissl_med[exposure_change_slices[1],
                                            :, 0]>0))) / annotation.shape[1], 
            color=CBcdict["Or"], linewidth=6)

plt.axvline(x=[exposure_change_slices[2]], 
            ymin=(np.max(np.where(nissl_med[exposure_change_slices[2],
                                            :, 0]>0)) + 30) / annotation.shape[1], 
            color=CBcdict["bG"], linewidth=6)
plt.axvline(x=[exposure_change_slices[2]], 
            ymax=(np.min(np.where(nissl_med[exposure_change_slices[2],
                                            :, 0]>0)) + 25) / annotation.shape[1], 
            color=CBcdict["bG"], linewidth=6)

plt.axis('off')
plt.xlim([0, annotation.shape[0]])
plt.ylim([annotation.shape[1], 0])
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "sagittal_nissl_median.png"), dpi=200)

In [ ]:
fig = plt.figure(figsize=(15, 7))
plt.axvline(x=[exposure_change_slices[0]],ymin=0, 
            color=CBcdict["Bu"], linewidth=6)
plt.axvline(x=[240],ymin=0, color=CBcdict["Or"], linewidth=6)
plt.axvline(x=[360],ymin=0, color=CBcdict["bG"], linewidth=6)
plt.plot(x_pos, hist_nissl, c='grey', linewidth=3)
plt.xlabel("Coronal section position")
plt.ylabel("Median Nissl value")
max_ = np.max(hist_nissl)
plt.xlim([0, annotation.shape[0]])
plt.ylim([0, max_])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.title("Evolution of Nissl median value in Whole Brain")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "distrib_nissl_median.png"), dpi=200)

### Process median nissl value for isocortex

In [ ]:
filt = np.isin(annotation, list(region_map.find("Isocortex", 
                                                attr="name", 
                                                with_descendants=True)))
filt = np.where(filt)
x_pos = np.unique(filt[0])

hist_nissl = np.zeros(len(x_pos))
for i, x in enumerate(x_pos):
    subfilt = np.where(filt[0]==x)
    hist_nissl[i]= np.median(nissl[x, filt[1][subfilt], 
                                   filt[2][subfilt]])

In [ ]:
fig = plt.figure(figsize=(15, 7))
plt.axvline(x=[exposure_change_slices[0]],ymin=0, 
            color=CBcdict["Bu"], linewidth=6)
plt.axvline(x=[exposure_change_slices[1]],ymin=0, 
            color=CBcdict["Or"], linewidth=6)
plt.axvline(x=[exposure_change_slices[2]],ymin=0, 
            color=CBcdict["bG"], linewidth=6)
plt.plot(x_pos, hist_nissl, c='grey', linewidth=3)
plt.xlabel("Coronal section position")
plt.ylabel("Median Nissl value")
plt.xlim([0, annotation.shape[0]])
max_ = np.max(hist_nissl)
plt.ylim([0, max_])
plt.ticklabel_format(style='sci', axis='y', scilimits=(0,0))
plt.title("Evolution of Nissl median value in Isocortex")
plt.tight_layout(pad=0)
plt.savefig(join(OUTPUT_FOLDER, "distrib_nissl_median_ctx.png"), dpi=200)